In [133]:
import pandas as pd
import numpy as np
import json

from secrets.paths import alle_co_path

In [100]:
assess_df = pd.read_csv(f'{alle_co_path}/assessments.csv')

/var/folders/6q/n3pxz9m53xq52fgdhf0cbl8h0000gn/T/ipykernel_80350/2200263208.py:1: DtypeWarning: Columns (19,20,29,30,31,37,38,46,57,83) have mixed types. Specify dtype option on import or set low_memory=False.
  assess_df = pd.read_csv(f'{alle_co_path}/assessments.csv')


In [101]:
# check PARID is unique
print(len(assess_df['PARID']) - len(assess_df['PARID'].unique()))

0


In [102]:
# convert integer columns with NaN (float) to pandas nullable integers
maybe_float_cols = assess_df.columns[assess_df.dtypes == 'float64']
true_float_cols = []
for col in maybe_float_cols:
  temp_ser = assess_df[col].dropna()
  n_floats = len(temp_ser[temp_ser != temp_ser.round(0)])
  if n_floats > 0:
    print(col, n_floats)
  else:
    true_float_cols.append(col)

assess_df[true_float_cols] = assess_df[true_float_cols].astype(pd.Int64Dtype())
print(assess_df.dtypes.unique())
# (assess_df[float_cols] != assess_df[float_cols].round()).sum()

STORIES 31544
[dtype('O') dtype('int64') Int64Dtype() dtype('float64')]


In [104]:
# clean up strings
obj_cols = assess_df.columns[assess_df.dtypes == 'O']
for col in obj_cols:
  # strip leading and trailing whitespace
  assess_df[col] = assess_df[col].str.strip()
  # replace multiple whitespace with single space
  # pandas regex doesn't like pd.NA
  assess_df[col].fillna('', inplace=True)
  assess_df[col] = assess_df[col].str.replace(r'\s\s+', ' ', regex=True)
  # replace empty strings with NA
  assess_df.loc[assess_df[col] == '', col] = pd.NA
  # paranoid (unnessecary?) replace only whitepace string with NA
  assess_df.loc[assess_df[col].str.match(r'^\s+$', na=True), col] = pd.NA

In [119]:
# probably unnecessary
assess_df.replace(np.nan, pd.NA, inplace=True)

In [115]:
assess_dict = {
  row['PARID']: row.drop('PARID').dropna().to_dict()
  for idx, row in assess_df.iloc[:10000].iterrows()
}

In [128]:
# inspired by https://stackoverflow.com/questions/60280466/merging-two-dataframes-with-pd-na-in-merge-column-yields-typeerror-boolean-val
# takes about 80s on my laptop
assess_dict = {
  m['PARID']: {
    k:v for k,v in m.items()
    if not pd.isna(v) and k != 'PARID'
  }
  for m in assess_df.to_dict('records')
}

In [132]:
print(len(assess_dict) == len(assess_df))

True


In [134]:
with open('assessments.json', 'w') as f:
    json.dump(assess_dict, f)

In [135]:
assess_df.to_feather('assessments_clean.feather')